In [11]:
#Помогло создание виртуального окружения
# !pip install numpy numba vispy
# !pip install jupyter_rfb

In [10]:
import numpy as np
from numba import njit

from vispy import gloo, app
from vispy.visuals import TextVisual

# Constants
num_boids = 100
width, height = 800, 600
size = 10
max_speed = 5

# Initialize boids
boids = np.random.uniform(0, 1, (num_boids, 2))
velocities = np.random.uniform(-1, 1, (num_boids, 2))

# Numba-compiled function to update boids positions
@njit(parallel=True)
def update_boids(boids, velocities):
    # Update boids positions
    for i in range(num_boids):
        boids[i] += velocities[i]
        
# Create a Vispy canvas
canvas = app.Canvas(keys='interactive', size=(width, height), title='Boids Simulation')
text = TextVisual('', color='white', pos=(10, 10))
text.font_size = 14

# Vispy shader program for rendering boids
vertex = """
attribute vec2 position;
void main() {
    gl_Position = vec4(position, 0.0, 1.0);
}
"""

fragment = """
void main() {
    gl_FragColor = vec4(1.0, 1.0, 1.0, 1.0);
}
"""

program = gloo.Program(vertex, fragment)
program['position'] = np.zeros((num_boids, 2), dtype=np.float32)

@canvas.connect
def on_draw(event):
    canvas.clear()
    update_boids(boids, velocities)
    program['position'] = boids * np.array([width, height])
    program.draw('points')
    text.text = f'Number of boids: {num_boids}\nMax speed: {max_speed}\nFPS: {canvas.measure_fps()}'
    text.draw(canvas.contexts[0])

if __name__ == '__main__':
    canvas.show(run=True)


RuntimeError: Could not import backend "jupyter_rfb":
The jupyter_rfb backend relies on a the jupyter_rfb library: ``pip install jupyter_rfb``